<a   href="https://colab.research.google.com/github//N-Nieto/Inner_Speech_Dataset/blob/master/Database_load_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial for load the Inner speech database.

## Set up - Download and import required libraries

In [15]:
!pip3 install mne -q

In [16]:
#@title Imports 
import mne 
import warnings
import numpy as np



from Python_Processing.Data_extractions import  extract_data_from_subject
from Python_Processing.Data_processing import  select_time_window, transform_for_classificator

np.random.seed(23)

mne.set_log_level(verbose='warning') #to avoid info at terminal
warnings.filterwarnings(action = "ignore", category = DeprecationWarning ) 
warnings.filterwarnings(action = "ignore", category = FutureWarning ) 

## Data Loading.

In [17]:
!pip3 install awscli

In [ ]:
!aws s3 ls --no-sign-request --recursive s3://openneuro.org/ds003626/

2021-04-17 19:51:45        125 ds003626/.datalad/.gitattributes
2021-04-17 19:51:46         63 ds003626/.datalad/config
2021-04-17 19:51:46        284 ds003626/.gitattributes
2022-03-14 21:25:02        377 ds003626/CHANGES
2022-03-14 21:25:02       1675 ds003626/README
2021-04-17 19:51:45         36 ds003626/annex-uuid
2022-03-14 21:25:02        647 ds003626/dataset_description.json
2021-04-17 19:51:48    4240275 ds003626/derivatives/sub-01/ses-01/sub-01_ses-01_baseline-epo.fif
2021-04-17 19:51:49  236165355 ds003626/derivatives/sub-01/ses-01/sub-01_ses-01_eeg-epo.fif
2022-03-14 21:16:17       6813 ds003626/derivatives/sub-01/ses-01/sub-01_ses-01_events.dat
2021-04-17 19:51:59   14775955 ds003626/derivatives/sub-01/ses-01/sub-01_ses-01_exg-epo.fif
2021-04-17 19:52:00        615 ds003626/derivatives/sub-01/ses-01/sub-01_ses-01_report.pkl
2021-04-17 19:52:00    4239417 ds003626/derivatives/sub-01/ses-02/sub-01_ses-02_baseline-epo.fif
2021-04-17 19:52:01  236164497 ds003626/derivatives/su

In [18]:
!aws s3 sync --no-sign-request s3://openneuro.org/ds003626 ds003626-download/

^C


In [ ]:
### Hyperparameters

# The root dir has to point to the folder that contains the database
root_dir = "/workspaces/Inner_Speech_Dataset/ds003626-download/"

# Data Type
datatype = "EEG"

# Sampling rate
fs = 256

# Select the useful par of each trial. Time in seconds
t_start = 1.5
t_end = 3.5

# Subject number
N_S = 1   #[1 to 10]


In [ ]:
#@title Data extraction and processing

# Load all trials for a single subject
X, Y = extract_data_from_subject(root_dir, N_S, datatype)

# Cut useful time. i.e action interval
X = select_time_window(X = X, t_start = t_start, t_end = t_end, fs = fs)

In [19]:
print("Data shape: [trials x channels x samples]")
print(X.shape) # Trials, channels, samples

print("Labels shape")
print(Y.shape) # Time stamp, class , condition, session

Data shape: [trials x channels x samples]
(500, 128, 512)
Labels shape
(500, 4)


array([0., 1.])

In [42]:
!python3 ./Python_Processing/Plotting/Plot_ERPs.py

LOL Current directory: /workspaces/Inner_Speech_Dataset
Traceback (most recent call last):
  File "./Python_Processing/Plotting/Plot_ERPs.py", line 22, in <module>
    from Data_extractions import extract_block_data_from_subject
  File "/workspaces/Inner_Speech_Dataset/Python_Processing/Data_extractions.py", line 14, in <module>
    from Python_Processing.Utilitys import sub_name, unify_names       # noqa
ModuleNotFoundError: No module named 'Python_Processing'


In [31]:
!ls

 Database_load_Tutorial.ipynb  'Stimulation Protocol - MatLab'
 Inner_Speech_Dataset	        ds003626-download
 LICENSE		        environment.yml
 Python_Processing	        oryx-build-commands.txt
 README.md		        venv


## Create the different groups for a classifier. A group is created with one condition and one class. 

In [20]:
# Conditions to compared
Conditions = [["Inner"],["Inner"]]
# The class for the above condition
Classes    = [  ["Up"] ,["Down"] ]

In [21]:
# Transform data and keep only the trials of interes
X , Y =  transform_for_classificator(X, Y, Classes, Conditions)

In [22]:
print("Final data shape")
print(X.shape)

print("Final labels shape")
print(Y.shape) 

Final data shape
(100, 128, 512)
Final labels shape
(100,)


In [28]:
X

array([[[ 1.65462687e-05,  1.64649149e-05,  1.20498564e-05, ...,
         -7.97352502e-07,  4.48656999e-06,  1.44397579e-05],
        [ 1.71068496e-05,  1.64661493e-05,  1.19874070e-05, ...,
         -9.93840837e-07,  5.72378655e-06,  1.58112060e-05],
        [ 1.73783707e-05,  1.64587800e-05,  1.19444289e-05, ...,
         -9.96159441e-09,  6.24076327e-06,  1.56441659e-05],
        ...,
        [ 1.81377665e-05,  6.25881032e-06, -1.93792653e-06, ...,
         -7.86858107e-06, -3.60174593e-06, -8.45177733e-07],
        [ 2.05662223e-05,  9.50295798e-06,  2.63608791e-06, ...,
         -8.15330652e-06, -4.30845362e-06, -3.09200181e-06],
        [ 2.11959667e-05,  9.47883264e-06,  1.75059815e-06, ...,
         -8.10794150e-06, -4.34914249e-06, -2.31489193e-06]],

       [[-7.03463288e-06, -1.49910685e-05, -9.31665882e-06, ...,
         -8.74191961e-06, -1.15206878e-05, -1.49518726e-05],
        [-8.06810115e-06, -1.36188294e-05, -9.93089818e-06, ...,
         -8.73303696e-06, -1.17522042e

In [27]:
np.unique(Y)

array([0., 1.])